In [1]:
from tfrbm.bbrbm import BBRBM

In [2]:
r = BBRBM(n_visible=2,n_hidden=4)

In [3]:
import numpy as np

In [4]:
x = np.array([[0,1],[1,0]])

In [9]:
for i in range(100):
    r.fit(x)

Epoch: 0
Train error: 0.2397

Epoch: 1
Train error: 0.2396

Epoch: 2
Train error: 0.2395

Epoch: 3
Train error: 0.2394

Epoch: 4
Train error: 0.2392

Epoch: 5
Train error: 0.2391

Epoch: 6
Train error: 0.2390

Epoch: 7
Train error: 0.2389

Epoch: 8
Train error: 0.2387

Epoch: 9
Train error: 0.2386

Epoch: 0
Train error: 0.2385

Epoch: 1
Train error: 0.2383

Epoch: 2
Train error: 0.2382

Epoch: 3
Train error: 0.2381

Epoch: 4
Train error: 0.2380

Epoch: 5
Train error: 0.2378

Epoch: 6
Train error: 0.2377

Epoch: 7
Train error: 0.2375

Epoch: 8
Train error: 0.2374

Epoch: 9
Train error: 0.2372

Epoch: 0
Train error: 0.2371

Epoch: 1
Train error: 0.2369

Epoch: 2
Train error: 0.2368

Epoch: 3
Train error: 0.2366

Epoch: 4
Train error: 0.2364

Epoch: 5
Train error: 0.2363

Epoch: 6
Train error: 0.2361

Epoch: 7
Train error: 0.2359

Epoch: 8
Train error: 0.2357

Epoch: 9
Train error: 0.2355

Epoch: 0
Train error: 0.2353

Epoch: 1
Train error: 0.2352

Epoch: 2
Train error: 0.2350

Epoch: 3
T

Epoch: 4
Train error: 0.1997

Epoch: 5
Train error: 0.1995

Epoch: 6
Train error: 0.1994

Epoch: 7
Train error: 0.1993

Epoch: 8
Train error: 0.1992

Epoch: 9
Train error: 0.1991

Epoch: 0
Train error: 0.1990

Epoch: 1
Train error: 0.1989

Epoch: 2
Train error: 0.1988

Epoch: 3
Train error: 0.1987

Epoch: 4
Train error: 0.1985

Epoch: 5
Train error: 0.1984

Epoch: 6
Train error: 0.1983

Epoch: 7
Train error: 0.1982

Epoch: 8
Train error: 0.1981

Epoch: 9
Train error: 0.1980

Epoch: 0
Train error: 0.1979

Epoch: 1
Train error: 0.1977

Epoch: 2
Train error: 0.1976

Epoch: 3
Train error: 0.1975

Epoch: 4
Train error: 0.1974

Epoch: 5
Train error: 0.1973

Epoch: 6
Train error: 0.1971

Epoch: 7
Train error: 0.1970

Epoch: 8
Train error: 0.1969

Epoch: 9
Train error: 0.1968

Epoch: 0
Train error: 0.1967

Epoch: 1
Train error: 0.1966

Epoch: 2
Train error: 0.1964

Epoch: 3
Train error: 0.1963

Epoch: 4
Train error: 0.1962

Epoch: 5
Train error: 0.1961

Epoch: 6
Train error: 0.1960

Epoch: 7
T

Train error: 0.1668

Epoch: 8
Train error: 0.1667

Epoch: 9
Train error: 0.1666

Epoch: 0
Train error: 0.1664

Epoch: 1
Train error: 0.1663

Epoch: 2
Train error: 0.1662

Epoch: 3
Train error: 0.1660

Epoch: 4
Train error: 0.1659

Epoch: 5
Train error: 0.1658

Epoch: 6
Train error: 0.1657

Epoch: 7
Train error: 0.1655

Epoch: 8
Train error: 0.1654

Epoch: 9
Train error: 0.1653

Epoch: 0
Train error: 0.1651

Epoch: 1
Train error: 0.1650

Epoch: 2
Train error: 0.1649

Epoch: 3
Train error: 0.1647

Epoch: 4
Train error: 0.1646

Epoch: 5
Train error: 0.1645

Epoch: 6
Train error: 0.1643

Epoch: 7
Train error: 0.1642

Epoch: 8
Train error: 0.1641

Epoch: 9
Train error: 0.1639

Epoch: 0
Train error: 0.1638

Epoch: 1
Train error: 0.1636

Epoch: 2
Train error: 0.1635

Epoch: 3
Train error: 0.1634

Epoch: 4
Train error: 0.1632

Epoch: 5
Train error: 0.1631

Epoch: 6
Train error: 0.1630

Epoch: 7
Train error: 0.1628

Epoch: 8
Train error: 0.1627

Epoch: 9
Train error: 0.1626

Epoch: 0
Train erro

Epoch: 1
Train error: 0.1273

Epoch: 2
Train error: 0.1271

Epoch: 3
Train error: 0.1270

Epoch: 4
Train error: 0.1268

Epoch: 5
Train error: 0.1266

Epoch: 6
Train error: 0.1265

Epoch: 7
Train error: 0.1263

Epoch: 8
Train error: 0.1262

Epoch: 9
Train error: 0.1260

Epoch: 0
Train error: 0.1259

Epoch: 1
Train error: 0.1257

Epoch: 2
Train error: 0.1255

Epoch: 3
Train error: 0.1254

Epoch: 4
Train error: 0.1252

Epoch: 5
Train error: 0.1251

Epoch: 6
Train error: 0.1249

Epoch: 7
Train error: 0.1248

Epoch: 8
Train error: 0.1246

Epoch: 9
Train error: 0.1244

Epoch: 0
Train error: 0.1243

Epoch: 1
Train error: 0.1241

Epoch: 2
Train error: 0.1240

Epoch: 3
Train error: 0.1238

Epoch: 4
Train error: 0.1237

Epoch: 5
Train error: 0.1235

Epoch: 6
Train error: 0.1234

Epoch: 7
Train error: 0.1232

Epoch: 8
Train error: 0.1231

Epoch: 9
Train error: 0.1229

Epoch: 0
Train error: 0.1228

Epoch: 1
Train error: 0.1226

Epoch: 2
Train error: 0.1225

Epoch: 3
Train error: 0.1223

Epoch: 4
T

In [10]:
r.get_err(x)

0.10046771

In [11]:
r.x

<tf.Tensor 'Placeholder:0' shape=(?, 2) dtype=float32>

In [12]:
r.predict([[1,-1]],[1])

[[1 0]
 [1 1]]
[-3.177043  -2.7259178]
0


array([1, 0])